# Dueling Double DQN + PER - Continuar Entrenamiento (ACELERADO)

Este notebook permite:
1. ✅ Cargar un modelo Dueling DDQN + PER pre-entrenado
2. 🚀 **ACELERAR el aprendizaje** que ya está ocurriendo
3. 📈 Mejorar más rápido sin perder estabilidad

## 📊 Análisis de tu gráfica:
- Episodios 0-20K: Aprendizaje lento pero estable (~600-800 puntos)
- Episodios 20K-27K: **¡EXPLOSIÓN!** Mejora de 800 → 1800 puntos (125% mejora)
- Picos alcanzados: 5000-6000 puntos
- **Conclusión**: El modelo YA aprendió a jugar bien, ahora lo aceleramos

## ⚡ Estrategia de aceleración:
1. **Learning rate 2.5x más alto**: Aprende más rápido de cada experiencia
2. **Batch más grande**: Gradientes más estables, menos ruido
3. **Buffer más pequeño**: Olvida experiencias antiguas, enfoca en lo reciente
4. **Epsilon muy bajo**: 90% explotación (el modelo ya sabe jugar)
5. **PER alpha más alto**: Prioriza más los errores grandes (aprendizaje eficiente)

In [1]:
# Configuración de directorios
import os

DIRECTORIO_BASE = os.path.join(os.getcwd(), "resultados_entrenamiento")
os.makedirs(DIRECTORIO_BASE, exist_ok=True)

print("Directorio base:", DIRECTORIO_BASE)
print("Resultados se guardarán en:", os.path.abspath(DIRECTORIO_BASE))

Directorio base: /Users/isaackeitor/Desktop/Galaxian/resultados_entrenamiento
Resultados se guardarán en: /Users/isaackeitor/Desktop/Galaxian/resultados_entrenamiento


In [2]:
# Preprocesamiento del entorno Atari
import gymnasium as gym
import ale_py
import numpy as np
import cv2
from collections import deque


class EnvolturaPreprocesamiento(gym.Wrapper):
    def __init__(self, entorno_base, salto_cuadros=4, tam_pantalla=84, escala_grises=True):
        super().__init__(entorno_base)
        self.salto_cuadros = salto_cuadros
        self.tam_pantalla = tam_pantalla
        self.escala_grises = escala_grises
        forma_obs = (tam_pantalla, tam_pantalla)
        if not escala_grises:
            forma_obs += (3,)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=forma_obs, dtype=np.uint8)

    def procesar_cuadro(self, cuadro):
        if self.escala_grises:
            cuadro = cv2.cvtColor(cuadro, cv2.COLOR_RGB2GRAY)
            cuadro = cv2.resize(cuadro, (self.tam_pantalla, self.tam_pantalla), interpolation=cv2.INTER_AREA)
            return cuadro
        else:
            cuadro = cv2.resize(cuadro, (self.tam_pantalla, self.tam_pantalla), interpolation=cv2.INTER_AREA)
            return cuadro

    def step(self, accion):
        recompensa_acumulada = 0.0
        terminado = truncado = False
        for _ in range(self.salto_cuadros):
            observacion_raw, recomp, term, trunc, informacion = self.env.step(accion)
            recompensa_acumulada += recomp
            terminado |= term
            truncado |= trunc
            if terminado or truncado:
                break
        cuadro_procesado = self.procesar_cuadro(observacion_raw)
        return cuadro_procesado, recompensa_acumulada, terminado, truncado, informacion

    def reset(self, **kwargs):
        observacion_raw, informacion = self.env.reset(**kwargs)
        cuadro_procesado = self.procesar_cuadro(observacion_raw)
        return cuadro_procesado, informacion


class EnvolturaApilamiento(gym.Wrapper):
    def __init__(self, entorno_base, num_apilar=4):
        super().__init__(entorno_base)
        self.num_apilar = num_apilar
        self.cuadros_memoria = deque([], maxlen=num_apilar)
        bajo = np.repeat(entorno_base.observation_space.low[np.newaxis, ...], num_apilar, axis=0)
        alto = np.repeat(entorno_base.observation_space.high[np.newaxis, ...], num_apilar, axis=0)
        self.observation_space = gym.spaces.Box(
            low=bajo.min(), high=alto.max(), dtype=entorno_base.observation_space.dtype,
            shape=(num_apilar, *entorno_base.observation_space.shape)
        )

    def reset(self, **kwargs):
        observacion, informacion = self.env.reset(**kwargs)
        for _ in range(self.num_apilar):
            self.cuadros_memoria.append(observacion)
        return self._obtener_obs(), informacion

    def step(self, accion):
        observacion, recompensa, terminado, truncado, informacion = self.env.step(accion)
        self.cuadros_memoria.append(observacion)
        return self._obtener_obs(), recompensa, terminado, truncado, informacion

    def _obtener_obs(self):
        return np.stack(self.cuadros_memoria, axis=0)


def crear_entorno_galaxian(semilla=None, modo_render=None):
    entorno = gym.make("ALE/Galaxian-v5", render_mode=modo_render)
    if semilla is not None:
        entorno.reset(seed=semilla)
    entorno = EnvolturaPreprocesamiento(entorno, salto_cuadros=4, tam_pantalla=84, escala_grises=True)
    entorno = EnvolturaApilamiento(entorno, num_apilar=4)
    return entorno

print("✅ Wrappers de preprocesamiento cargados")

✅ Wrappers de preprocesamiento cargados


In [3]:
# Implementación Dueling Double DQN + PER
import os
import csv
import random
from typing import Tuple, List

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# Detectar dispositivo
if torch.backends.mps.is_available():
    dispositivo = torch.device("mps")
    print("✅ Usando GPU de Apple Silicon (MPS)")
elif torch.cuda.is_available():
    dispositivo = torch.device("cuda")
    print("✅ Usando GPU CUDA")
else:
    dispositivo = torch.device("cpu")
    print("⚠️ Usando CPU")


# ===========================
#  Árboles de Segmentos para PER
# ===========================
class ArbolSegmentos:
    def __init__(self, capacidad, funcion_reduccion):
        assert capacidad > 0 and (capacidad & (capacidad - 1)) == 0, "Capacidad debe ser potencia de 2"
        self.capacidad = capacidad
        self.arbol = np.zeros(2 * capacidad, dtype=np.float32)
        self.funcion_reduccion = funcion_reduccion

    def actualizar(self, indice, valor):
        i = indice + self.capacidad
        self.arbol[i] = valor
        i //= 2
        while i >= 1:
            self.arbol[i] = self.funcion_reduccion(self.arbol[2 * i], self.arbol[2 * i + 1])
            i //= 2

    def reducir(self, inicio, fin):
        resultado_izq = None
        resultado_der = None
        inicio += self.capacidad
        fin += self.capacidad
        while inicio <= fin:
            if (inicio % 2) == 1:
                resultado_izq = self.arbol[inicio] if resultado_izq is None else self.funcion_reduccion(resultado_izq, self.arbol[inicio])
                inicio += 1
            if (fin % 2) == 0:
                resultado_der = self.arbol[fin] if resultado_der is None else self.funcion_reduccion(self.arbol[fin], resultado_der)
                fin -= 1
            inicio //= 2
            fin //= 2
        if resultado_izq is None:
            return resultado_der
        if resultado_der is None:
            return resultado_izq
        return self.funcion_reduccion(resultado_izq, resultado_der)

    def __getitem__(self, indice):
        return self.arbol[indice + self.capacidad]


class ArbolSuma(ArbolSegmentos):
    def __init__(self, capacidad):
        super().__init__(capacidad, funcion_reduccion=lambda a, b: a + b)

    def suma_total(self):
        return self.arbol[1]

    def encontrar_suma_prefijo(self, suma_objetivo):
        idx = 1
        while idx < self.capacidad:
            izquierda = 2 * idx
            if self.arbol[izquierda] >= suma_objetivo:
                idx = izquierda
            else:
                suma_objetivo -= self.arbol[izquierda]
                idx = izquierda + 1
        return idx - self.capacidad


class ArbolMinimo(ArbolSegmentos):
    def __init__(self, capacidad):
        super().__init__(capacidad, funcion_reduccion=min)

    def minimo_total(self):
        return self.arbol[1]


class BufferReplayPriorizado:
    def __init__(self, capacidad_maxima: int, alfa_prioridad: float = 0.6, epsilon_per: float = 1e-6):
        potencia_2 = 1
        while potencia_2 < capacidad_maxima:
            potencia_2 *= 2
        self.capacidad = potencia_2
        self.alfa_prioridad = alfa_prioridad
        self.epsilon_per = epsilon_per
        self.posicion = 0
        self.tamanio = 0
        self.estados = [None] * self.capacidad
        self.acciones = np.zeros(self.capacidad, dtype=np.int64)
        self.recompensas = np.zeros(self.capacidad, dtype=np.float32)
        self.estados_siguientes = [None] * self.capacidad
        self.terminados = np.zeros(self.capacidad, dtype=np.bool_)
        self.arbol_suma = ArbolSuma(self.capacidad)
        self.arbol_minimo = ArbolMinimo(self.capacidad)
        self.prioridad_maxima = 1.0
        for i in range(self.capacidad):
            self.arbol_suma.actualizar(i, 0.0)
            self.arbol_minimo.actualizar(i, float("inf"))

    def __len__(self):
        return self.tamanio

    def agregar(self, estado, accion, recompensa, estado_sig, terminal):
        idx = self.posicion
        self.estados[idx] = estado
        self.acciones[idx] = accion
        self.recompensas[idx] = recompensa
        self.estados_siguientes[idx] = estado_sig
        self.terminados[idx] = terminal
        prioridad = (self.prioridad_maxima + self.epsilon_per) ** self.alfa_prioridad
        self.arbol_suma.actualizar(idx, prioridad)
        self.arbol_minimo.actualizar(idx, prioridad)
        self.posicion = (self.posicion + 1) % self.capacidad
        self.tamanio = min(self.tamanio + 1, self.capacidad)

    def muestrear(self, tam_lote: int, beta_importancia: float = 0.4):
        indices_salida = []
        estados_salida = []
        acciones_salida = np.empty(tam_lote, dtype=np.int64)
        recompensas_salida = np.empty(tam_lote, dtype=np.float32)
        estados_sig_salida = []
        terminados_salida = np.empty(tam_lote, dtype=np.float32)
        suma_total = self.arbol_suma.suma_total()
        segmento = suma_total / tam_lote
        probabilidad_minima = self.arbol_minimo.minimo_total() / suma_total
        peso_maximo = (probabilidad_minima * self.tamanio) ** (-beta_importancia)
        for i in range(tam_lote):
            a = segmento * i
            b = segmento * (i + 1)
            masa = random.random() * (b - a) + a
            idx = self.arbol_suma.encontrar_suma_prefijo(masa)
            indices_salida.append(idx)
            estados_salida.append(self.estados[idx])
            acciones_salida[i] = self.acciones[idx]
            recompensas_salida[i] = self.recompensas[idx]
            estados_sig_salida.append(self.estados_siguientes[idx])
            terminados_salida[i] = float(self.terminados[idx])
        probabilidades = np.array([self.arbol_suma[idx] / suma_total for idx in indices_salida], dtype=np.float32)
        pesos_is = (probabilidades * self.tamanio) ** (-beta_importancia)
        pesos_is = pesos_is / peso_maximo
        pesos_is = pesos_is.astype(np.float32)
        return np.array(indices_salida), pesos_is, (np.array(estados_salida), acciones_salida, recompensas_salida, np.array(estados_sig_salida), terminados_salida)

    def actualizar_prioridades(self, indices, prioridades):
        for idx, prioridad in zip(indices, prioridades):
            prioridad = float(prioridad + self.epsilon_per)
            self.arbol_suma.actualizar(idx, prioridad ** self.alfa_prioridad)
            self.arbol_minimo.actualizar(idx, prioridad ** self.alfa_prioridad)
            self.prioridad_maxima = max(self.prioridad_maxima, prioridad)


class RedDuelingDQN(nn.Module):
    def __init__(self, forma_entrada: Tuple[int, int, int], num_acciones: int):
        super().__init__()
        canales, alto, ancho = forma_entrada
        self.canales_esperados = canales
        self.extractor_caracteristicas = nn.Sequential(
            nn.Conv2d(canales, 32, kernel_size=8, stride=4),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.Flatten()
        )
        with torch.no_grad():
            tam_aplanado = self.extractor_caracteristicas(torch.zeros(1, canales, alto, ancho)).shape[1]
        self.stream_valor = nn.Sequential(
            nn.Linear(tam_aplanado, 512), nn.ReLU(inplace=True),
            nn.Linear(512, 1)
        )
        self.stream_ventaja = nn.Sequential(
            nn.Linear(tam_aplanado, 512), nn.ReLU(inplace=True),
            nn.Linear(512, num_acciones)
        )

    def forward(self, tensor_entrada):
        if tensor_entrada.ndim != 4:
            raise ValueError(f"tensor_entrada.ndim={tensor_entrada.ndim}, esperado 4")
        if tensor_entrada.shape[1] != self.canales_esperados and tensor_entrada.shape[-1] == self.canales_esperados:
            tensor_entrada = tensor_entrada.permute(0, 3, 1, 2)
        tensor_entrada = tensor_entrada.float() / 255.0
        caracteristicas = self.extractor_caracteristicas(tensor_entrada)
        valor_estado = self.stream_valor(caracteristicas)
        ventaja_acciones = self.stream_ventaja(caracteristicas)
        valores_q = valor_estado + (ventaja_acciones - ventaja_acciones.mean(dim=1, keepdim=True))
        return valores_q


def _calcular_media_movil(valores: List[float], ventana: int = 100):
    if len(valores) == 0:
        return []
    resultado = []
    suma_acum = 0.0
    cola = []
    for v in valores:
        cola.append(v)
        suma_acum += v
        if len(cola) > ventana:
            suma_acum -= cola.pop(0)
        resultado.append(suma_acum / len(cola))
    return resultado


def _guardar_grafica_y_csv(dir_checkpoints: str, recompensas: List[float], episodio: int):
    os.makedirs(dir_checkpoints, exist_ok=True)
    ruta_csv = os.path.join(dir_checkpoints, "registro_recompensas.csv")
    archivo_nuevo = not os.path.exists(ruta_csv)
    with open(ruta_csv, "a", newline="") as archivo:
        escritor = csv.writer(archivo)
        if archivo_nuevo:
            escritor.writerow(["episodio", "recompensa"])
        escritor.writerow([episodio, recompensas[-1]])
    plt.figure(figsize=(10, 5))
    plt.plot(recompensas, label="Recompensa", color="#FF6B35", linewidth=1.2, alpha=0.7)
    media_movil = _calcular_media_movil(recompensas, ventana=100)
    if len(media_movil) > 0:
        plt.plot(media_movil, label="Media Móvil (100)", color="#00A896", linewidth=2.5)
    plt.xlabel("Episodio", fontsize=12)
    plt.ylabel("Recompensa Total", fontsize=12)
    plt.title("Progreso - Dueling DDQN + PER", fontsize=14, fontweight="bold")
    plt.legend(loc="upper left")
    plt.grid(True, alpha=0.3)
    ruta_png = os.path.join(dir_checkpoints, f"recompensas_ep{episodio}.png")
    plt.tight_layout()
    plt.savefig(ruta_png, dpi=120)
    plt.close()
    print(f"[REGISTRO] Gráfica guardada: {ruta_png}")


print("✅ Clases Dueling DDQN + PER cargadas")

✅ Usando GPU de Apple Silicon (MPS)
✅ Clases Dueling DDQN + PER cargadas


In [4]:
# Función para continuar entrenamiento Dueling DDQN + PER

def continuar_entrenamiento_ddqn_per(
    ruta_checkpoint: str,
    directorio_checkpoints: str,
    episodios_adicionales: int = 2000,
    capacidad_buffer: int = 100_000,
    tam_lote: int = 32,
    factor_descuento: float = 0.99,
    tasa_aprendizaje: float = 1e-4,
    epsilon_inicial: float = 0.15,
    epsilon_final: float = 0.05,
    episodios_decaimiento_eps: int = 1000,
    intervalo_actualizacion_target: int = 1000,
    pasos_inicio_entrenamiento: int = 5_000,
    alfa_per: float = 0.6,
    beta_per_inicial: float = 0.4,
    beta_per_final: float = 1.0,
    epsilon_per: float = 1e-6,
    intervalo_guardado: int = 200,
    intervalo_graficas: int = 200,
    semilla_aleatoria: int = 42,
):
    os.makedirs(directorio_checkpoints, exist_ok=True)
    
    # 1. CARGAR CHECKPOINT
    print("\n" + "="*70)
    print("📂 CARGANDO CHECKPOINT DDQN+PER")
    print("="*70)
    checkpoint = torch.load(ruta_checkpoint, map_location=dispositivo, weights_only=False)
    
    episodio_inicio = checkpoint['episodio']
    pasos_globales_inicio = checkpoint['pasos_globales']
    recompensas_previas = checkpoint['recompensas']
    forma_entrada = tuple(checkpoint['forma_entrada'])
    num_acciones = checkpoint['num_acciones']
    
    recompensa_promedio = np.mean(recompensas_previas[-100:]) if len(recompensas_previas) >= 100 else np.mean(recompensas_previas)
    recompensa_maxima = max(recompensas_previas)
    
    print(f"✅ Checkpoint cargado: episodio {episodio_inicio}")
    print(f"📊 Recompensa promedio últimos 100 eps: {recompensa_promedio:.1f}")
    print(f"🏆 Recompensa máxima: {recompensa_maxima:.1f}")
    print(f"🎯 Pasos totales: {pasos_globales_inicio:,}")
    print("="*70 + "\n")
    
    # 2. CREAR ENTORNO
    entorno_juego = crear_entorno_galaxian(semilla=semilla_aleatoria, modo_render=None)
    
    # 3. RECREAR REDES
    red_q_principal = RedDuelingDQN(forma_entrada, num_acciones).to(dispositivo)
    red_q_objetivo = RedDuelingDQN(forma_entrada, num_acciones).to(dispositivo)
    red_q_principal.load_state_dict(checkpoint['red_q'])
    red_q_objetivo.load_state_dict(checkpoint['red_objetivo'])
    red_q_objetivo.eval()
    
    # 4. RECREAR OPTIMIZADOR
    optimizador = optim.Adam(red_q_principal.parameters(), lr=tasa_aprendizaje)
    optimizador.load_state_dict(checkpoint['optimizador'])
    
    # 5. CREAR NUEVO BUFFER PER
    memoria_experiencias = BufferReplayPriorizado(capacidad_buffer, alfa_prioridad=alfa_per, epsilon_per=epsilon_per)
    
    # 6. CONFIGURACIÓN
    pasos_globales = pasos_globales_inicio
    registro_recompensas = recompensas_previas.copy()
    episodio_final = episodio_inicio + episodios_adicionales
    episodios_annealing_beta = episodios_adicionales
    
    def calcular_epsilon(ep_actual: int, ep_inicio: int) -> float:
        ep_relativo = ep_actual - ep_inicio
        if ep_relativo >= episodios_decaimiento_eps:
            return epsilon_final
        fraccion = ep_relativo / float(episodios_decaimiento_eps)
        return epsilon_inicial + fraccion * (epsilon_final - epsilon_inicial)
    
    def calcular_beta(ep_actual: int, ep_inicio: int) -> float:
        ep_relativo = ep_actual - ep_inicio
        fraccion = min(1.0, ep_relativo / float(episodios_annealing_beta))
        return beta_per_inicial + fraccion * (beta_per_final - beta_per_inicial)
    
    # 7. MOSTRAR CONFIGURACIÓN
    print("="*70)
    print("🚀 INICIANDO ENTRENAMIENTO CONTINUO - DDQN+PER")
    print("="*70)
    print(f"📍 Episodio inicial: {episodio_inicio + 1}")
    print(f"🎯 Episodio final: {episodio_final}")
    print(f"➕ Episodios adicionales: {episodios_adicionales}")
    print(f"\n🔧 HIPERPARÁMETROS ORIGINALES:")
    print(f"   Buffer: {capacidad_buffer:,}")
    print(f"   Batch: {tam_lote}")
    print(f"   Learning rate: {tasa_aprendizaje}")
    print(f"   Gamma: {factor_descuento}")
    print(f"   Epsilon: {epsilon_inicial} → {epsilon_final}")
    print(f"   Target update: cada {intervalo_actualizacion_target} pasos")
    print(f"   PER alpha: {alfa_per}")
    print(f"   PER beta: {beta_per_inicial} → {beta_per_final}")
    print("="*70 + "\n")
    
    # 8. BUCLE DE ENTRENAMIENTO
    for episodio_actual in range(episodio_inicio + 1, episodio_final + 1):
        observacion, _ = entorno_juego.reset()
        finalizado = False
        recompensa_total = 0.0
        epsilon_actual = calcular_epsilon(episodio_actual, episodio_inicio)
        beta_actual = calcular_beta(episodio_actual, episodio_inicio)
        pasos_en_episodio = 0
        
        while not finalizado:
            pasos_globales += 1
            pasos_en_episodio += 1
            
            # Política epsilon-greedy
            if random.random() < epsilon_actual:
                accion_elegida = entorno_juego.action_space.sample()
            else:
                with torch.no_grad():
                    obs_lote = np.expand_dims(observacion, axis=0)
                    obs_tensor = torch.from_numpy(obs_lote).to(dispositivo)
                    valores_q = red_q_principal(obs_tensor)
                    accion_elegida = int(torch.argmax(valores_q, dim=1).item())
            
            siguiente_obs, recompensa_step, terminado, truncado, _ = entorno_juego.step(accion_elegida)
            finalizado = terminado or truncado
            recompensa_total += float(recompensa_step)
            
            memoria_experiencias.agregar(observacion, accion_elegida, recompensa_step, siguiente_obs, finalizado)
            observacion = siguiente_obs
            
            # Entrenamiento
            if len(memoria_experiencias) >= pasos_inicio_entrenamiento:
                indices, pesos_is, lote = memoria_experiencias.muestrear(tam_lote, beta_importancia=beta_actual)
                estados, acciones, recompensas, estados_sig, terminados = lote
                
                estados_t = torch.from_numpy(estados).to(dispositivo)
                estados_sig_t = torch.from_numpy(estados_sig).to(dispositivo)
                acciones_t = torch.from_numpy(acciones).long().to(dispositivo)
                recompensas_t = torch.from_numpy(recompensas).float().to(dispositivo)
                terminados_t = torch.from_numpy(terminados).float().to(dispositivo)
                pesos_is_t = torch.from_numpy(pesos_is).float().to(dispositivo)
                
                valores_q = red_q_principal(estados_t).gather(1, acciones_t.unsqueeze(1)).squeeze(1)
                
                # Double DQN
                with torch.no_grad():
                    q_principal_sig = red_q_principal(estados_sig_t)
                    acciones_optimas = torch.argmax(q_principal_sig, dim=1, keepdim=True)
                    q_objetivo_sig = red_q_objetivo(estados_sig_t)
                    q_siguientes = q_objetivo_sig.gather(1, acciones_optimas).squeeze(1)
                    q_objetivo_valores = recompensas_t + factor_descuento * q_siguientes * (1.0 - terminados_t)
                
                errores_td = q_objetivo_valores - valores_q
                perdida = (pesos_is_t * errores_td.pow(2)).mean()
                
                optimizador.zero_grad()
                perdida.backward()
                nn.utils.clip_grad_norm_(red_q_principal.parameters(), 10.0)
                optimizador.step()
                
                nuevas_prioridades = errores_td.detach().abs().cpu().numpy() + epsilon_per
                memoria_experiencias.actualizar_prioridades(indices, nuevas_prioridades)
            
            # Actualizar red objetivo
            if pasos_globales % intervalo_actualizacion_target == 0:
                red_q_objetivo.load_state_dict(red_q_principal.state_dict())
        
        registro_recompensas.append(recompensa_total)
        promedio_100 = np.mean(registro_recompensas[-100:]) if len(registro_recompensas) >= 100 else np.mean(registro_recompensas)
        
        print(f"[DDQN+PER] Ep {episodio_actual}/{episodio_final} | R: {recompensa_total:.1f} | "
              f"Prom100: {promedio_100:.1f} | ε={epsilon_actual:.3f} | β={beta_actual:.3f} | Mem={len(memoria_experiencias)}")
        
        # Guardar checkpoint
        if episodio_actual % intervalo_guardado == 0:
            ruta_ckpt = os.path.join(directorio_checkpoints, f"ddqn_per_ep{episodio_actual}.pth")
            torch.save({
                "red_q": red_q_principal.state_dict(),
                "red_objetivo": red_q_objetivo.state_dict(),
                "optimizador": optimizador.state_dict(),
                "episodio": episodio_actual,
                "pasos_globales": pasos_globales,
                "recompensas": registro_recompensas,
                "forma_entrada": forma_entrada,
                "num_acciones": num_acciones,
            }, ruta_ckpt)
            print(f"[CHECKPOINT] Guardado: {ruta_ckpt}")
        
        # Guardar gráficas
        if episodio_actual % intervalo_graficas == 0:
            _guardar_grafica_y_csv(directorio_checkpoints, registro_recompensas, episodio_actual)
    
    entorno_juego.close()
    
    # Guardar modelo final
    ruta_final = os.path.join(directorio_checkpoints, f"ddqn_per_final_ep{episodio_final}.pth")
    torch.save(red_q_principal.state_dict(), ruta_final)
    
    print("\n" + "="*70)
    print("🎉 ENTRENAMIENTO COMPLETADO")
    print("="*70)
    print(f"✅ Modelo final: {ruta_final}")
    print(f"📊 Episodios totales: {episodio_final}")
    print(f"🏆 Mejor recompensa: {max(registro_recompensas):.1f}")
    print(f"📈 Promedio últimos 100 eps: {np.mean(registro_recompensas[-100:]):.1f}")
    print("="*70 + "\n")
    
    return red_q_principal


print("✅ Función de entrenamiento continuo DDQN+PER cargada")

✅ Función de entrenamiento continuo DDQN+PER cargada


## 🎯 Instrucciones:

### 1. Encuentra tu último checkpoint DDQN+PER
Busca en `resultados_entrenamiento/ddqn_per_continuado/` el checkpoint más reciente (ep27000 según tu gráfica)

### 2. Verifica la ruta
La celda siguiente ya está configurada para cargar `ddqn_per_ep27000.pth`

### 3. Ejecuta y monitorea
- **Espera ver**: Media móvil subiendo de ~1800 a ~2500-3000 en 3000 episodios
- **Si la media móvil cae más de 20%**: Detén el entrenamiento, los hiperparámetros son muy agresivos
- **Si sube constantemente**: ¡Perfecto! Puedes extender otros 2000-3000 episodios después

### 4. Resultados esperados:
- Episodio 28000: ~2000 puntos
- Episodio 29000: ~2400 puntos
- Episodio 30000: ~2800+ puntos

In [15]:
# ========================================
# CONFIGURACIÓN: ACELERACIÓN DE APRENDIZAJE
# ========================================

# 1. RUTA DEL CHECKPOINT
if "DIRECTORIO_BASE" not in globals():
    DIRECTORIO_BASE = os.path.join(os.getcwd(), "resultados_entrenamiento")
    os.makedirs(DIRECTORIO_BASE, exist_ok=True)

# Carga el checkpoint del último episodio (27000 según la gráfica)
RUTA_CHECKPOINT = os.path.join(DIRECTORIO_BASE, "ddqn_per_acelerado/ddqn_per_ep24800.pth")

if not os.path.exists(RUTA_CHECKPOINT):
    print(f"⚠️ ERROR: No se encontró: {RUTA_CHECKPOINT}")
    print(f"\n📁 Archivos disponibles:")
    if os.path.exists(os.path.dirname(RUTA_CHECKPOINT)):
        for archivo in sorted(os.listdir(os.path.dirname(RUTA_CHECKPOINT))):
            if archivo.endswith('.pth'):
                print(f"   - {archivo}")
else:
    print(f"✅ Checkpoint encontrado: {RUTA_CHECKPOINT}")
    
    DIRECTORIO_NUEVO = os.path.join(DIRECTORIO_BASE, "ddqn_per_acelerado")
    
    print("\n" + "="*70)
    print("🚀 CONFIGURACIÓN: APRENDIZAJE ACELERADO - DDQN+PER")
    print("="*70)
    print("🎯 El modelo YA está mejorando (600→1800 puntos)")
    print("⚡ Aceleramos el aprendizaje sin romper estabilidad")
    print("="*70 + "\n")
    
    # ENTRENAR CON APRENDIZAJE ACELERADO
    red_mejorada = continuar_entrenamiento_ddqn_per(
        ruta_checkpoint=RUTA_CHECKPOINT,
        directorio_checkpoints=DIRECTORIO_NUEVO,
        
        episodios_adicionales=10000,          # 3000 episodios adicionales
        
        # ⚡⚡ HIPERPARÁMETROS ACELERADOS ⚡⚡
        capacidad_buffer=80_000,             # ⬇️ Más pequeño = aprende de experiencias recientes
        tam_lote=64,                         # ⬆️ Más grande = gradientes más estables
        factor_descuento=0.99,               # ✓ Mantener
        tasa_aprendizaje=2.5e-4,             # ⬆️⬆️ 2.5x MÁS RÁPIDO (antes 1e-4)
        intervalo_actualizacion_target=1200, # ⬆️ Un poco más lento para estabilidad
        alfa_per=0.7,                        # ⬆️ Más énfasis en errores grandes
        beta_per_inicial=0.5,                # ⬆️ Más corrección IS desde el inicio
        beta_per_final=1.0,                  # ✓ Mantener
        epsilon_per=1e-6,                    # ✓ Mantener
        
        # Exploración muy baja (el modelo ya sabe jugar)
        epsilon_inicial=0.10,                # ⬇️⬇️ Muy bajo (90% greedy)
        epsilon_final=0.02,                  # ⬇️⬇️ Casi greedy puro
        episodios_decaimiento_eps=1000,      # Rápido decay
        pasos_inicio_entrenamiento=3_000,    # ⬇️ Empieza antes
        
        intervalo_guardado=200,
        intervalo_graficas=200,
        semilla_aleatoria=42,
    )
    
    print("\n" + "="*70)
    print("📊 CAMBIOS CLAVE PARA ACELERAR:")
    print("="*70)
    print("⚡ Learning rate: 1e-4 → 2.5e-4  (2.5x más rápido)")
    print("⚡ Batch size: 32 → 64  (gradientes más estables)")
    print("⚡ Buffer: 100K → 80K  (olvida experiencias antiguas)")
    print("⚡ Epsilon: 0.15 → 0.10  (más explotación)")
    print("⚡ PER alpha: 0.6 → 0.7  (prioriza más errores grandes)")
    print("="*70)

✅ Checkpoint encontrado: /Users/isaackeitor/Desktop/Galaxian/resultados_entrenamiento/ddqn_per_acelerado/ddqn_per_ep24800.pth

🚀 CONFIGURACIÓN: APRENDIZAJE ACELERADO - DDQN+PER
🎯 El modelo YA está mejorando (600→1800 puntos)
⚡ Aceleramos el aprendizaje sin romper estabilidad


📂 CARGANDO CHECKPOINT DDQN+PER
✅ Checkpoint cargado: episodio 24800
📊 Recompensa promedio últimos 100 eps: 1435.6
🏆 Recompensa máxima: 5780.0
🎯 Pasos totales: 4,226,136

🚀 INICIANDO ENTRENAMIENTO CONTINUO - DDQN+PER
📍 Episodio inicial: 24801
🎯 Episodio final: 34800
➕ Episodios adicionales: 10000

🔧 HIPERPARÁMETROS ORIGINALES:
   Buffer: 80,000
   Batch: 64
   Learning rate: 0.00025
   Gamma: 0.99
   Epsilon: 0.1 → 0.02
   Target update: cada 1200 pasos
   PER alpha: 0.7
   PER beta: 0.5 → 1.0

[DDQN+PER] Ep 24801/34800 | R: 2090.0 | Prom100: 1449.4 | ε=0.100 | β=0.500 | Mem=548
[DDQN+PER] Ep 24802/34800 | R: 2130.0 | Prom100: 1456.7 | ε=0.100 | β=0.500 | Mem=958
[DDQN+PER] Ep 24803/34800 | R: 1060.0 | Prom100: 14

KeyboardInterrupt: 